# Generate Customer Personas



### Overview


Using OpenAI API for GPT-3 genterate customer personas.

Content generation is a hot topic and a viable usecase for Large Language Models LLMs like GPT.


### Free Scripts

This is part of a series of free python scripts. 

You can find all the scripts online at https://github.com/FrontAnalyticsInc/data-winners

Message me on twitter if you have any suggestions.


### About Me

My name is Alton Alexander. I am a Data Science consultant turned entreprenuer building SaaS tools for SEO.

Find more about my free scripts or ask me any question on twitter: @alton_lex

# Setup

Steps: 

1) You'll need an API key from OpenAI.

2) Paste the key in the cell below or add it as an environment variable. For best practices see https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety

3) Run the following locally or in a google colab environment

In [27]:
# install libraries
%pip install openai

# load libraries
import openai
from pprint import pprint

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Set your api key
# copy and paste your open api key as a varible

openai.api_key = "sk-PASTEYOURKEYHERE"

# Create a function

Inputs: the prompt

Outputs: the text-generated

In [10]:
def openai_textgen_for_datawinners(
        prompt,
        model = "text-davinci-003",
        max_length = 4000
    ):
    """ A generic wrapper for the endpoint """
    
    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      temperature=0.7,
      max_tokens=max_length,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return response.get('choices',[])[0]['text']

In [93]:

# we build variables
prompt_audience = "data winners"

# we build a prompt template (note the f in front of string!)
prompt = f"""Say how cool this is to all the {prompt_audience}"""

prompt

'Say how cool this is to all the data winners'

In [28]:
# test the function

prompt = "Say how cool this is to all the data winners"
openai_textgen_for_datawinners(prompt)

'\n\nCongratulations to all the data winners! This is such an awesome accomplishment and shows the power of data to answer questions and solve problems. You all should be very proud of yourselves!'

# Primitive customer idea

Lets say you run a travel website and want to personalize the content for different travelers

## List buyer personas

In [13]:
# describe your target customer generally
target = "consumers who are interested in learning more about domestic travel destinations"
list_n = 5

prompt = f"""[Target: {target}]

Please give me {list_n} buyer personas:"""

response = openai_textgen_for_datawinners(prompt)

In [50]:
print(prompt)
print(response)

[Target: consumers who are interested in learning more about domestic travel destinations]

Please give me 5 buyer personas:
 

1. Working Professional - A professional aged 25-35 who wants to take a break from the daily grind and explore new cities and attractions. 

2. Adventure Seeker - A young adult aged 18-25 who is looking to explore the great outdoors, take part in adrenaline-filled activities, and meet new people. 

3. Retiree - A retired individual aged 55+ who is looking to travel within the US and learn more about the country's history and culture. 

4. Family Traveler - A family with children aged 3-18 looking to explore the US with fun and educational activities. 

5. Road Tripper - A young adult or working professional aged 18-35 who enjoys taking road trips to explore different places and take in the scenery.


In [39]:
# split the response into a dictionary
personas = {}

for each_response in response.split("\n"):
    try:
        title = each_response.split(".",1)[1].split("-")[0].strip()
        description = each_response.split(".",1)[1].split("-",1)[1].strip()
        personas[ title ] = {
            'title':title,
            'description':description.replace("\n"," ")
        }
        
        print("### ", title)
        print()
        print(personas[title])
        print()
    except:
        True


###  Working Professional

{'title': 'Working Professional', 'description': 'A professional aged 25-35 who wants to take a break from the daily grind and explore new cities and attractions.'}

###  Adventure Seeker

{'title': 'Adventure Seeker', 'description': 'A young adult aged 18-25 who is looking to explore the great outdoors, take part in adrenaline-filled activities, and meet new people.'}

###  Retiree

{'title': 'Retiree', 'description': "A retired individual aged 55+ who is looking to travel within the US and learn more about the country's history and culture."}

###  Family Traveler

{'title': 'Family Traveler', 'description': 'A family with children aged 3-18 looking to explore the US with fun and educational activities.'}

###  Road Tripper

{'title': 'Road Tripper', 'description': 'A young adult or working professional aged 18-35 who enjoys taking road trips to explore different places and take in the scenery.'}



## Build a persona profile

Now we can create a series of prompts to explore each persona, one a time.

In [54]:

def create_profile( persona ):
    
    title = persona['title']
    description = persona['description']
    
    # no name yet but we will create one!
    name = persona.get('name', "nameless")
    gender = persona.get('gender', "unknown")
    groupsize = persona.get('groupsize', None)
    
    questions = {
        "name": f"In one word what is the firstname of the person who is a {title} and who is best described as {description}",
        "gender": f"in one word (male or female) what is the gender of a person named %(name)s who is a {title} and who is best described as {description}",
        "groupsize": f"in one number, give the average size of group where you would find a {title} who is best described as {description}",
        "pain": f"list examples of pains (psychological, spiritul, or physical) felt by a {title} who is best described as {description}",
        "insecurities": f"list what humiliates %(name)s who is a {title} who is best described as {description}",
        "books": f"list what books does %(name)s who is a {title} read if they are best described as {description}",
        "movies": f"list what movies and tv shows does %(name)s watch if they are a {title} and they are best described as {description}",
        "desires": f"list what %(name)s secretly desire as a {title} and especially as it is related to being {description}",
        "socialmedia": f"list which forums, chat rooms, facebook groups, reddit channels, etc that %(name)s frequently visits as a {title} and especially as it is related to being {description}",
        "leadmagents": f"list of lead magnets for captivating {target} especially ones that are a {title} and who are described being {description}"
    }
    
    for qtype in questions:
        
        # create the prompt with special responses
        prompt = questions[qtype] % {"name":name, "gender":gender, "groupsize":groupsize}
        response = openai_textgen_for_datawinners(prompt)
        
        # save the response
        persona[qtype] = response
        
        # special responses
        if qtype == 'name':
            name = response.strip().strip("\n").strip().strip(".")
        if qtype == 'gender':
            gender = response.strip().strip("\n").strip().strip(".")
        if qtype == 'groupsize':
            groupsize = response.strip().strip("\n").strip().strip(".")
            
        print("#### ", qtype)
        print(response)
        print()
        print()

        
# test with one
create_profile( personas['Family Traveler'] )
        

####  name


Rebecca


####  gender


Female.


####  groupsize


Average group size: 6


####  pain


Psychological: 
- Fear of missing out on key tourist attractions 
- Anxiety over finding the right places to visit 
- Stress of planning the trip 

Spiritual: 
- Feeling overwhelmed by the amount of choices 
- Feeling disconnected from family members due to the busyness of the trip 
- Feeling lost in unfamiliar surroundings 

Physical: 
- Muscle fatigue from carrying luggage 
- Lack of sleep from long hours of traveling 
- Blisters from walking too much


####  insecurities
 

1. Having to cut short an exciting family trip due to lack of money or time.
2. Not being able to afford expensive entertainment, such as theme parks or shows.
3. Not being able to visit all of the desired destinations.
4. Not being able to explore as much as other families due to limited resources.
5. Not being able to find suitable accommodation or transportation.
6. Having to stay in budget hotels or campgrou

## Repeat for each of the personas

In [55]:

create_profile( personas['Adventure Seeker'] )

####  name


Ethan.


####  gender


Male


####  groupsize


4-5


####  pain


Psychological: Fear of failure, feeling unsure of one’s abilities in new situations, worry about the future, stress, anxiety

Spiritual: Feeling disconnected from the natural world, feeling disconnected from others, feeling lost, feeling a lack of purpose

Physical: Muscle strain, exhaustion, dehydration, sunburn, insect bites, sprains, cuts, bruises.


####  insecurities


1. Being told he isn't capable of completing an outdoor activity or adrenaline-filled activity.
2. Being asked to stay inside instead of going outdoors and exploring.
3. Being asked to participate in activities he isn't interested in.
4. Being asked to stay inside with family or friends when he wants to explore alone.
5. Being asked to participate in activities that do not involve any physical activity.
6. Being asked to stay away from people and places he wants to explore.
7. Being told he is too young to participate in activities he i

In [56]:

create_profile( personas['Working Professional'] )

####  name


Explorer


####  gender


Female.


####  groupsize


7.5


####  pain


Psychological Pains:
- Anxiety
- Stress
- Fear of Failure
- Loneliness

Spiritual Pains:
- Loss of Meaning
- Loss of Purpose
- Feeling Disconnected
- Feeling Unfulfilled

Physical Pains:
- Headaches
- Muscle Tension
- Fatigue
- Insomnia


####  insecurities


1. Not having a plan or a budget for the trip. 
2. Not doing enough research before the trip.
3. Not allowing enough time for planning and preparation.
4. Not taking enough time off from work.
5. Not having a backup plan in case something goes wrong.
6. Not bringing the right clothing or equipment.
7. Not taking enough money for the trip. 
8. Not having a travel insurance policy.
9. Not booking accommodations in advance.
10. Not reading up on the local culture and customs.


####  books


1. Lonely Planet World Guide: A Traveler's Guide to the World's Best Cities and Attractions
2. The Rough Guide to Europe
3. The Complete Guide to Budget Travel:

In [57]:

create_profile( personas['Retiree'] )

####  name


Explorer


####  gender


Male


####  groupsize


3


####  pain


Psychological:
- Fear of the unknown 
- Anxiety about the safety of their travels 
- Feelings of loneliness and isolation 
- Difficulty adjusting to a new routine and lifestyle

Spiritual:
- Uncertainty about the future 
- Feeling disconnected from their previous life 
- Loss of purpose

Physical:
- Aches and pains due to aging 
- Fatigue from travel 
- Joint pain from long days of sightseeing


####  insecurities


1. Being told you're too old to travel. 
2. Being told you don't have the capacity to learn. 
3. Not being able to keep up with the physical demands of a trip. 
4. Not being able to afford the cost of travel. 
5. Feeling out of place or not belonging in certain places. 
6. Not being able to communicate or understand local customs. 
7. Being ignored or overlooked by other travelers or locals. 
8. Being ridiculed for your lack of knowledge of the culture, history, or language. 
9. Having to ask f

## Initialize your own persona

Let OpenAI create the details

In [ ]:

create_profile( {
    "title": "solo traveler",
    "description": "a person who likes to travel on their own to various destinations"
} )

####  name


Soloist.


####  gender


Genderless.


####  groupsize


1


